In [1]:
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot') #style
matplotlib.rcParams['figure.figsize'] = (12, 5)#size graph
df = pd.read_hdf("./data.h5")

# print(df.shape)
# df.sample(5)

df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)
print(df_customers)

df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
df_customers.head(5)
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc
print('.....................................')
df_customers["most_revenue_customer"].value_counts(normalize=True) *100
print('................................')
print(df_customers[df_customers["most_revenue_customer"]]['customer_id'])
print('................................')
top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())
len(top_customers)
df["most_revenue_customer"] = df["customer_id"].map(lambda x: x in top_customers)
df["most_revenue_customer"].value_counts()

      customer_id  customer_price_total
448           448              61130939
907           907              53007160
2647         2647              33695110
179           179              31039769
66             66              29774259
...           ...                   ...
4594         4594                   295
180           180                   250
4286         4286                     0
4784         4784                     0
4785         4785                     0

[5879 rows x 2 columns]
.....................................
................................
448      448
907      907
2647    2647
179      179
66        66
        ... 
2142    2142
1550    1550
1584    1584
1135    1135
302      302
Name: customer_id, Length: 1324, dtype: int64
................................


True     539072
False    281834
Name: most_revenue_customer, dtype: int64

In [3]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GroupKFold

import gc

df = pd.read_hdf("./data.h5")

print(df.shape)
df.sample(5)

df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)

df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()

df_customers = (
    df
    .groupby("customer_id")
    .agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("product_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    ).reset_index()
)

df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)

df_customers
df_customers["most_revenue_customer"].value_counts(normalize=True)

feats = list(df_customers.select_dtypes([np.number, bool]).columns)

feats
black_list = ["most_revenue_customer"]
feats = [x for x in feats if x not in black_list]

X = df_customers[feats].values
y = df_customers["most_revenue_customer"].values

scores = cross_val_score(DecisionTreeClassifier(max_depth=3), X, y, scoring="accuracy", cv=5)
np.mean(scores), np.std(scores)

(820906, 9)


(0.9996598639455783, 0.0004165798882284457)